### functionality

### required directory structure

### imports and inputs

In [34]:
import pandas as pd
import seaborn as sb
import os
import matplotlib.pyplot as plt
from packages import plotstyle as ps
from packages import miscellaneous as mc
sb.set_theme(style="white",font='serif')

date = '8_3_2021 (evening)' # Since Archive is reserved for unprocessed solutions, we need to tap into an "Observational 
                   # Data" directory for our processed solution.
    
ymax = 0.003 # Manual override to help manage outliers. If not needed, set to 0.
    
directory = '/Users/colemeyer/Documents/ALMA Internship/Observational Data/' # Directory of "Observational Data", assuming specified structure

### loading and cleaning data

In [35]:
directory = directory+'/'+date

# Create new "errorplots" directory within "Observational Data" subdirectory
if not os.path.isdir(directory+'/baselineplots/'):
    os.mkdir(directory+'/baselineplots')
    
numObs = mc.numObs(directory)

### extrema

In [36]:
first = True
for file in os.listdir(directory+'/asdm'):
    if os.path.isdir(directory+'/asdm/'+file):
        antArr = pd.read_csv(directory+'/asdm/'+file+'/AntData.csv')[['xPhaseAvg','poserrX','poserrY','poserrZ','geoposX','geoposY','geoposZ']]
        
        # Calculate antenna separation distances
        distances = [0]
        for i in range(1,antArr.shape[0]):
            distances.append(mc.distance(antArr.iloc[0,4:7],antArr.iloc[i,4:7]))
        
        antArr['distances'] = distances
        antArr = antArr.iloc[1:]
        
        # Combine all data end-to-end
        if not first: 
            oldAntArr = oldAntArr.append(antArr)
        else: 
            oldAntArr = antArr
            first = False
            
# Take abs. value of position offset to display position offset magnitude
oldAntArr.loc[:,'poserrX':'poserrZ'] = abs(oldAntArr.loc[:,'poserrX':'poserrZ'])
            
# Set up plot limits
xMax = max(oldAntArr.loc[:,'poserrX'])
yMax = max(oldAntArr.loc[:,'poserrY'])
zMax = max(oldAntArr.loc[:,'poserrZ'])

xmin, xmax = 0, max(oldAntArr.loc[:,'distances'])*1.1
ymin = 0 
if ymax == 0: ymax = max(xMax,yMax,zMax)*1.1

# plot

In [37]:
plot = 0
for uid in os.listdir(directory+'/asdm'):
    if os.path.isdir(directory+'/asdm/'+uid):
        antArr = pd.read_csv(directory+'/asdm/'+uid+'/AntData.csv')[['xPhaseAvg','poserrX','poserrY','poserrZ','geoposX','geoposY','geoposZ']]
        
        # Calculate antenna separation distances
        distances = [0]
        for i in range(1,antArr.shape[0]):
            distances.append(mc.distance(antArr.iloc[0,4:7],antArr.iloc[i,4:7]))
        
        antArr['distances'] = distances
        antArr = antArr.iloc[1:]
        
        # Take abs. value of position offset to display position offset magnitude
        antArr.loc[:,'xPhaseAvg':'poserrZ'] = abs(antArr.loc[:,'xPhaseAvg':'poserrZ'])
        
        fig, axes = plt.subplots(1,3,figsize=(15,5),sharey=True)
        
        # Call sb.scatterplot for all three subplots
        sb.scatterplot(ax=axes[0], x='distances', y='poserrX', data=antArr, hue='xPhaseAvg', size='xPhaseAvg')
        sb.scatterplot(ax=axes[1], x='distances', y='poserrY', data=antArr, hue='xPhaseAvg', size='xPhaseAvg')
        sb.scatterplot(ax=axes[2], x='distances', y='poserrZ', data=antArr, hue='xPhaseAvg', size='xPhaseAvg')
        
        # Apply styling
        ps.plotNonDescript(axes[0],title='Ant.Sep. vs. Position Offset (x), *'+uid[11:],xmax=xmax,xmin=xmin,ymax=ymax,
                          ymin=ymin,titlesize=11,xlabel='Ant.Sep. Distance (m)',ylabel='Position Offset Magnitude (m)',
                           xticks=[],yticks=True)
        ps.plotNonDescript(axes[1],title='Ant.Sep. vs. Position Offset (y), *'+uid[11:],xmax=xmax,xmin=xmin,ymax=ymax,
                          ymin=ymin,titlesize=11,xlabel='Ant.Sep. Distance (m)', xticks=[],yticks=True)
        ps.plotNonDescript(axes[2],title='Ant.Sep. vs. Position Offset (z), *'+uid[11:],xmax=xmax,xmin=xmin,ymax=ymax,
                          ymin=ymin,titlesize=11,xlabel='Ant.Sep. Distance (m)', xticks=[],yticks=True)
        
        for i in range(3): axes[i].legend(title='Avg. Phase', fontsize=11, title_fontsize=11)
        
        plt.tight_layout()
        
        plot += 1
        
        #plt.show()
        plt.savefig(directory+'/baselineplots/baselineplots'+str(plot)+'.png', bbox_inches='tight')
        plt.close()